# Day 09. Exercise 02
# Metrics

## 0. Imports

In [321]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import joblib
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [322]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')

weeks = pd.read_csv('../data/dayofweek.csv')
df["weeks"] = weeks['dayofweek']


In [323]:
df.head()

,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,weeks
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4


In [324]:
X = df.drop(["weeks"],axis=1)
y = df["weeks"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [325]:
svc = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', random_state=21, probability=True)
svc.fit(X_train, y_train)

SVC(C=10, gamma='auto', probability=True, random_state=21)

In [326]:
y_pred = svc.predict(X_test)
y_scores = svc.predict_proba(X_test)

In [327]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)


y_test_bin = label_binarize(y_test, classes=np.unique(y))
y_scores_bin = label_binarize(np.argmax(y_scores, axis=1), classes=np.unique(y))

roc_auc = []
for i in range(y_test_bin.shape[1]):
    if np.sum(y_test_bin[:, i]) > 0: 
        roc_auc.append(roc_auc_score(y_test_bin[:, i], y_scores_bin[:, i]))

if roc_auc: 
    weighted_roc_auc = np.average(roc_auc, weights=[np.sum(y_test_bin[:, i]) for i in range(y_test_bin.shape[1]) if np.sum(y_test_bin[:, i]) > 0])
else:
    weighted_roc_auc = 0.0  

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')
print(f'roc_auc is {weighted_roc_auc:.5f}')

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.93822


## 3. Decision tree

1. The same task for decision tree

In [328]:
tree = DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth=25, random_state=21)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=25, random_state=21)

In [329]:
y_pred = tree.predict(X_test)
y_scores = tree.predict_proba(X_test)

In [330]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)


y_test_bin = label_binarize(y_test, classes=np.unique(y))
y_scores_bin = label_binarize(np.argmax(y_scores, axis=1), classes=np.unique(y))

roc_auc = roc_auc_score(y_test_bin, y_scores_bin)

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')
print(f'roc_auc is {roc_auc:.5f}')

accuracy is 0.90533
precision is 0.90667
recall is 0.90533
roc_auc is 0.93986


## 4. Random forest

1. The same task for random forest.

In [331]:
tree_rand = RandomForestClassifier(class_weight='balanced', criterion='gini', max_depth=22, n_estimators=100, random_state=21)
tree_rand.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=22, random_state=21)

In [332]:
y_pred = tree_rand.predict(X_test)
y_scores = tree_rand.predict_proba(X_test)

In [333]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)


y_test_bin = label_binarize(y_test, classes=np.unique(y))
y_scores_bin = label_binarize(np.argmax(y_scores, axis=1), classes=np.unique(y))

roc_auc = roc_auc_score(y_test_bin, y_scores_bin)

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')
print(f'roc_auc is {roc_auc:.5f}')

accuracy is 0.93491
precision is 0.93565
recall is 0.93491
roc_auc is 0.94926


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [334]:
best_model = tree_rand

In [335]:
X_test["predict_day"]=best_model.predict(X_test)
X_test["weeks"] = y_test
X_test["error"] = X_test["predict_day"] != X_test["weeks"]

In [336]:
err = X_test[X_test["error"]]

In [337]:
group_errr = err.groupby("weeks").size()

In [338]:
all_size = df.index.size
array_count = list(group_errr)
for i, current in enumerate(array_count):
    print(f"day {i}   {current/all_size*100:.3f}%")

day 0   0.415%
day 1   0.119%
day 2   0.119%
day 3   0.178%
day 4   0.178%
day 5   0.237%
day 6   0.059%


In [339]:
error_analysis = {
    'labname_errors': {},
    'user_errors': {}
}

In [340]:
for labname in X_test.columns[X_test.columns.str.startswith('labname_')]:
    total_samples = X_test[X_test[labname] == 1].shape[0]
    error_samples = X_test[(X_test[labname] == 1) & (X_test['error'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['labname_errors'][labname] = error_rate

In [341]:
error_analysis['labname_errors']

{'labname_code_rvw': 7.6923076923076925,
 'labname_lab02': 0,
 'labname_lab03': 100.0,
 'labname_lab03s': 100.0,
 'labname_lab05s': 16.666666666666664,
 'labname_laba04': 14.285714285714285,
 'labname_laba04s': 4.0,
 'labname_laba05': 2.127659574468085,
 'labname_laba06': 11.11111111111111,
 'labname_laba06s': 6.666666666666667,
 'labname_project1': 4.838709677419355}

In [342]:
for user_col in X_test.columns[X_test.columns.str.startswith('uid_user_')]:
    total_samples = X_test[X_test[user_col] == 1].shape[0]
    error_samples = X_test[(X_test[user_col] == 1) & (X_test['error'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['user_errors'][user_col] = error_rate

In [343]:
error_analysis['user_errors']

{'uid_user_0': 0,
 'uid_user_1': 0.0,
 'uid_user_10': 8.333333333333332,
 'uid_user_11': 0,
 'uid_user_12': 0.0,
 'uid_user_13': 5.88235294117647,
 'uid_user_14': 3.225806451612903,
 'uid_user_15': 0.0,
 'uid_user_16': 40.0,
 'uid_user_17': 0.0,
 'uid_user_18': 16.666666666666664,
 'uid_user_19': 10.526315789473683,
 'uid_user_2': 3.571428571428571,
 'uid_user_20': 0.0,
 'uid_user_21': 0.0,
 'uid_user_22': 0.0,
 'uid_user_23': 0.0,
 'uid_user_24': 9.090909090909092,
 'uid_user_25': 9.090909090909092,
 'uid_user_26': 0.0,
 'uid_user_27': 16.666666666666664,
 'uid_user_28': 0.0,
 'uid_user_29': 9.090909090909092,
 'uid_user_3': 14.285714285714285,
 'uid_user_30': 12.5,
 'uid_user_31': 11.11111111111111,
 'uid_user_4': 7.4074074074074066,
 'uid_user_6': 25.0,
 'uid_user_7': 0,
 'uid_user_8': 0.0}

In [344]:
joblib.dump(best_model, 'my_model_ex02.pkl')

['my_model_ex02.pkl']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [345]:
X_test = X_test.drop(["error","predict_day","weeks"],axis=1)

In [346]:
def eval_metrics(models, params, X_train, y_train, X_test, y_test, y):
    results = {}
    
    for model, param in zip(models, params):

        model_instance = model(**param)

        model_instance.fit(X_train, y_train)

        y_pred = model_instance.predict(X_test)
        y_scores = tree.predict_proba(X_test)

        y_test_bin = label_binarize(y_test, classes=np.unique(y))
        y_scores_bin = label_binarize(np.argmax(y_scores, axis=1), classes=np.unique(y))
        
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        roc_auc = roc_auc_score(y_test_bin, y_scores_bin)

        results[model.__name__] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'auc_score': roc_auc
        }
    
    return results

In [347]:
models = [DecisionTreeClassifier, RandomForestClassifier]
params = [{'criterion': 'gini', 'max_depth': 36}, {'n_estimators': 100}]
results = eval_metrics(models, params, X_train, y_train, X_test, y_test, y)
print(results)

{'DecisionTreeClassifier': {'accuracy': 0.8698224852071006, 'precision': 0.873917151946469, 'recall': 0.8698224852071006, 'auc_score': 0.9398561477347495}, 'RandomForestClassifier': {'accuracy': 0.9349112426035503, 'precision': 0.9373863068969585, 'recall': 0.9349112426035503, 'auc_score': 0.9398561477347495}}
